In [1]:
# Python SQL toolkit 
import pandas as pd
import json
from sqlalchemy import create_engine, func

In [2]:
# create engine
engine = create_engine('sqlite:///data.sqlite')

In [3]:
# Query all data from table
fire_data = pd.read_sql_query('SELECT * FROM "Fires"', con=engine)

In [4]:
# https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
def df_to_geojson(df, properties, lat='LATITUDE', lon='LONGITUDE'):
    # create a new python dict to contain our geojson data, using geojson format
    gjs = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for index, row in fire_data.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        gjs['features'].append(feature)
    
    return gjs

In [5]:
# Define the properties of each feature, excluding LATITUDE and LONGITUDE as that is in the geometry section
fire_props = fire_data.loc[:, ~fire_data.columns.isin(['LATITUDE', 'LONGITUDE'])]
cols = fire_props.columns.values.tolist()


geojson = df_to_geojson(fire_data, cols)

In [7]:
output_file=open("output/Fires.geojson", "w", encoding="utf-8")
json.dump(geojson, output_file)


output_file.close()